# Convert GOM data to CDT-grid

In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.tri as mtri
from scipy.spatial import Delaunay
import scipy.interpolate
import collections
collections.Iterable = collections.abc.Iterable
collections.Mapping = collections.abc.Mapping
import ipywidgets as ipw
#import ipyvolume as ipv
import os

I keep the data separate from the code - the `schubtaltex.csv` file is kept in my home
directory 

`~/simdb/data/`

Adapt the path to the relevant location using `os.path.join`

In [ ]:
import os.path as op
input_dir = op.join(
    op.expanduser('~'), 'simdb', 'data', 'shear_zone', 'B1_TV1', 'dic_fine_data'
)
input_dir

# Read all DIC states into a list of arrays

In [ ]:
pxyz_files = [op.join(input_dir, each)
         for each in sorted(os.listdir(input_dir))
         if each.endswith('.csv')]

In [ ]:
pxyz_list = [
    np.loadtxt(csv_file, dtype=np.float_,
               skiprows=6, delimiter=';')
    for csv_file in pxyz_files
]

# Identify the points that are included in all time steps.

In [ ]:
P_list = [ np.array(pxyz[:,0], dtype=np.int_)
    for pxyz in pxyz_list
]

In [ ]:
max_n_P = np.max(np.array([ np.max(P_) for P_ in P_list ])) + 1

In [ ]:
P_Q = P_list[0] 
for P_next in P_list[1:]:
    P_Q = np.intersect1d(P_Q, P_next)
P_Q

# Define the initial configuration

In [ ]:
n_T = len(pxyz_list)
X_TPa = np.zeros((n_T, max_n_P, 3), dtype=np.float_)
for T in range(n_T):
    X_TPa[T, P_list[T]] = pxyz_list[T][:,1:]

In [ ]:
U_TPa = np.zeros_like(X_TPa)
for T in range(1, n_T):
    U_TPa[T, P_Q] = X_TPa[T, P_Q] - X_TPa[0, P_Q]

In [ ]:
X_Qa = X_TPa[0, P_Q]
U_TQa = U_TPa[:, P_Q]

# Add corner points

Set up a rectangular domain around the measured data.
Currently, the values on the border are set to 1.0, which is not 
useful for cases when the border should stop at a certain cut through 
the boundaries. In such cases, the data values in $z$ direction 
along the boundary triangle edges should be projected onto the 
boundary of the bounding box. This has not yet been done here.

In [ ]:
min_x, min_y, min_z = np.min(X_Qa, axis=0)
max_x, max_y, max_z = np.max(X_Qa, axis=0)

In [ ]:
argmin_x, argmax_x = np.argmin(X_Qa[:,0]), np.argmax(X_Qa[:,0])
argmin_y, argmax_y = np.argmin(X_Qa[:,1]), np.argmax(X_Qa[:,1])
argmin_z, argmax_z = np.argmin(X_Qa[:,2]), np.argmax(X_Qa[:,2])
argmin_x, argmax_x, argmin_y, argmax_y

In [ ]:
arg_ll = X_Qa[:,0] * X_Qa[:,1]

In [ ]:
X_0 = X_Qa[argmin_x, :]
X_1 = X_Qa[argmax_x, :]
X_2 = X_Qa[argmin_y, :]
V_0 = X_1 - X_0
V_1 = X_2 - X_0
v_0 = V_0 / np.linalg.norm(V_0)
v_1 = V_1 / np.linalg.norm(V_1)
EPS = np.zeros((3, 3, 3), dtype='f')
EPS[(0, 1, 2), (1, 2, 0), (2, 0, 1)] = 1
EPS[(2, 1, 0), (1, 0, 2), (0, 2, 1)] = -1
V_3 = np.einsum('abc,b,c->a', EPS, v_0, v_1)
v_3 = V_3 / np.linalg.norm(V_3)
v_3

# Construct the surface

Take the data along the first two axes, use the `trisurf` method of `Delaunay` to make an $xy$ triangulation

In [ ]:
points = X_Qa[:, :-1]
values = U_TQa[15, :, 1]
delaunay = Delaunay(points)
triangles=delaunay.simplices

In [ ]:
triangles.shape

# Plot using matplotlib with 3d projection
This version is static

In [ ]:
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(1, 1, 1, projection='3d')
#ax = fig.add_subplot(1, 1, 1)
x, y = points.T
z = values
ax.plot_trisurf(x, y, z, triangles=triangles, cmap=plt.cm.Spectral)
ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$y$')
ax.set_zlabel(r'$z$')
#ax.axis('equal')
ax.view_init(20, 20)
# ax.set_zlim(0,1);

# Define the grid

In [ ]:
d_x = max_x - min_x
d_y = max_y - min_y
d_x, d_y
boundary = d_y * 0.1

In [ ]:
delta_x, delta_y = 1, 1 # [mm]

In [ ]:
n_I = int(d_x / delta_x)
n_J = int(d_y / delta_y)
n_I, n_J

In [ ]:
X_aIJ = np.mgrid[min_x+boundary:max_x-boundary:complex(n_I),
                 min_y+boundary:max_y-boundary:complex(n_J)]
x_IJ, y_IJ = X_aIJ

# Interpolate the values on the grid

In [ ]:
points = X_Qa[:, :-1]
delaunay = Delaunay(points)
triangles=delaunay.simplices
values = U_TQa[16, :, :]
get_U = scipy.interpolate.LinearNDInterpolator(delaunay, values)

In [ ]:
get_U(40, 40)

In [ ]:
points = X_Qa[:, :-1]
delaunay = Delaunay(points)
triangles=delaunay.simplices
U_IJa_list = []
for T in range(n_T):
    values = U_TQa[T, :, :]
    get_U = scipy.interpolate.LinearNDInterpolator(delaunay, values)
    U_IJa = get_U(x_IJ, y_IJ)
    U_IJa_list.append(U_IJa)
U_TIJa = np.array(U_IJa_list)

In [ ]:
U_TIJa.shape

# Plot the values

In [ ]:
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(1, 1, 1, projection='3d')
z_IJ = U_TIJa[-4,...,1]
surf = ax.plot_surface(x_IJ, y_IJ, z_IJ, cmap=plt.cm.coolwarm,
                       linewidth=0, antialiased=False)

# Customize the z axis.
#ax.set_zlim(0, 1.01)
ax.zaxis.set_major_locator(plt.LinearLocator(10))
ax.zaxis.set_major_formatter(plt.FormatStrFormatter('%.02f'))

# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=5);
